In [16]:
import numpy as np
import pandas as pd
import rasterio
import rioxarray as rxr
import xarray as xr
import os
import warnings
warnings.filterwarnings("ignore")

In [17]:
date = pd.to_datetime('01-01-2020')
date

Timestamp('2020-01-01 00:00:00')

In [18]:
def read_land_cover(hdf_path):
    # Read as DataArrays with a "band" dimension of length up to 4
    aod = rxr.open_rasterio(hdf_path)['LC_Type1']
    #qa  = rxr.open_rasterio(sds(hdf_path, "AOD_QA"),             masked=True).astype("uint16")

    # Scale AOD (INT16 * 0.001). Keep shape [band, y, x].
    #aod = aod * 0.001

    # Apply QA per band
    #good = mask_from_qa(qa.values)  # shape [band, y, x]
    #aod.data = np.where(good, aod.values, np.nan)

    # Reduce across multiple overpasses (bands) to a single daily raster (median is robust)
    # (xarray names the dim "band" by default)
    #aod_daily = aod.median(dim="band")#, skipna=True)

    # Ensure NoData is NaN
    aod_daily = aod.rename("LC")
    #aod_daily.rio.write_nodata(np.nan, inplace=True)
    return aod_daily

In [19]:
def mosaic_and_reproject(list_of_dss):
    # Mosaic in native CRS
    '''
    Mosaic multiple DataArrays by averaging overlapping pixels.
    Assumes all DataArrays have the same CRS and resolution.
    list_if_dss: list of xarray.DataArray
    returns: xarray.DataArray
    '''
    stack = xr.concat(list_of_dss, dim="stack")
    mosaic = stack.mean("stack", skipna=True)

    # Reproject to WGS84 (≈1 km at equator = 30 arcsec)
    mosaic_wgs84 = mosaic.rio.reproject(
        "EPSG:4326",
        resolution=0.01, #0.00926625433
        resampling=rasterio.enums.Resampling.nearest,
        nodata=np.nan
    )
    return mosaic_wgs84

In [20]:
#ok
def convert_names_to_dates(files):
    year = []
    day = []
    #hr = []
    #min = []
    dts = []
    for file in files:
        dts.append(file[9:16])
        year.append(int(file[9:13]))
        day.append(int(file[13:16]))
        #hr.append(int(file[18:20]))
        #min.append(int(file[20:22]))
    files = pd.DataFrame(files, columns=['Files'])
    dts = pd.DataFrame(dts, columns=['MODIS_Dates'])
    year = pd.DataFrame(year, columns=['Year'])
    day = pd.DataFrame(day, columns=['Day'])
    #hr = pd.DataFrame(hr, columns=['Hour'])
    #min = pd.DataFrame(min, columns=['Minute'])
    dtdf = pd.concat([files, dts, year, day], axis=1)
    dtdf['datetime'] = pd.to_datetime(dtdf['Year'] * 1000 + dtdf['Day'], format='%Y%j')
    dtdf.sort_values(by=['datetime'], inplace=True, ascending=True)
    dtdf.reset_index(drop=True, inplace=True)
    return dtdf

In [21]:
convert_names_to_dates(os.listdir('./Land Cover Type'))

,Files,MODIS_Dates,Year,Day,datetime
0,MCD12Q1.A2020001.h20v05.061.2022171223657.hdf,2020001,2020,1,2020-01-01
1,MCD12Q1.A2020001.h21v05.061.2022171231100.hdf,2020001,2020,1,2020-01-01
2,MCD12Q1.A2021001.h20v05.061.2022216064217.hdf,2021001,2021,1,2021-01-01
3,MCD12Q1.A2021001.h21v05.061.2022216142720.hdf,2021001,2021,1,2021-01-01
4,MCD12Q1.A2022001.h20v05.061.2023243212239.hdf,2022001,2022,1,2022-01-01
5,MCD12Q1.A2022001.h21v05.061.2023244035412.hdf,2022001,2022,1,2022-01-01
6,MCD12Q1.A2023001.h20v05.061.2024252130142.hdf,2023001,2023,1,2023-01-01
7,MCD12Q1.A2023001.h21v05.061.2024252130508.hdf,2023001,2023,1,2023-01-01
8,MCD12Q1.A2024001.h20v05.061.2025205175754.hdf,2024001,2024,1,2024-01-01
9,MCD12Q1.A2024001.h21v05.061.2025205222418.hdf,2024001,2024,1,2024-01-01


In [22]:
def hdf_to_netcdf(folder, minx, miny, maxx, maxy):
    files = os.listdir(folder)
    df = convert_names_to_dates(files)
    for date in df['datetime'].unique():
        dff = df[df['datetime'] == date]
        dss = [read_land_cover(folder + file) for file in dff['Files'] if file.endswith('.hdf')]
        s = mosaic_and_reproject(dss).rio.clip_box(minx=minx-1e-16, miny=miny-1e-16, maxx=maxx+1e-16, maxy=maxy+1e-16)
        s['x'] = np.round(s['x'], 2)
        s['y'] = np.round(s['y'], 2)
        ds_date = s.to_dataset().expand_dims(time=[date]).assign_coords(time=("time", [date]))
        if date == df['datetime'].unique()[0]:
            final_ds = ds_date
        else:
            final_ds = xr.concat([final_ds, ds_date], dim='time')

    return final_ds

In [23]:
minx=35.5
miny=31.0
maxx=37.4
maxy=32.2

In [24]:
os.listdir('./Land Cover Type/')

['MCD12Q1.A2020001.h20v05.061.2022171223657.hdf',
 'MCD12Q1.A2020001.h21v05.061.2022171231100.hdf',
 'MCD12Q1.A2021001.h20v05.061.2022216064217.hdf',
 'MCD12Q1.A2021001.h21v05.061.2022216142720.hdf',
 'MCD12Q1.A2022001.h20v05.061.2023243212239.hdf',
 'MCD12Q1.A2022001.h21v05.061.2023244035412.hdf',
 'MCD12Q1.A2023001.h20v05.061.2024252130142.hdf',
 'MCD12Q1.A2023001.h21v05.061.2024252130508.hdf',
 'MCD12Q1.A2024001.h20v05.061.2025205175754.hdf',
 'MCD12Q1.A2024001.h21v05.061.2025205222418.hdf']

In [25]:
folder = './Land Cover Type/'

In [27]:
nc = hdf_to_netcdf(folder, minx, miny, maxx, maxy)
df = nc.to_dataframe().reset_index().drop(['band', 'spatial_ref'], axis=1)
df['latitude'] = df['y']
df['longitude'] = df['x']
df.drop(['x', 'y'], axis=1, inplace=True)
df.set_index(['time', 'latitude', 'longitude'], inplace=True)
df['LC'] = np.int8(np.round(df['LC'], 0))
nc = df.to_xarray()
nc

<xarray.Dataset> Size: 118kB
Dimensions:    (time: 5, latitude: 121, longitude: 191)
Coordinates:
  * time       (time) datetime64[ns] 40B 2020-01-01 2021-01-01 ... 2024-01-01
  * latitude   (latitude) float64 968B 31.0 31.01 31.02 ... 32.18 32.19 32.2
  * longitude  (longitude) float64 2kB 35.5 35.51 35.52 ... 37.38 37.39 37.4
Data variables:
    LC         (time, latitude, longitude) int8 116kB 16 16 16 16 ... 16 16 16

In [28]:
times = []
for i in range(len(nc['time'])):
    yr=nc['time'][i].dt.year.values
    times.append(pd.Series(pd.date_range(start=f'{yr}-01-01', end=f'{yr}-12-31', freq='D'), name='time'))

In [31]:
ncs =[]
times = []
ncdts = nc['time']
start = pd.to_datetime('12-10-2020')
end = pd.to_datetime('20-01-2025')
for i in range(len(ncdts)):
    yr=ncdts[i].dt.year.values
    if yr == start.year:
        times.append(pd.Series(pd.date_range(start=str(start)[0:10], end=f'{yr}-12-31', freq='D'), 
                           name='time'))
    elif end.year - yr == 1:
        times.append(pd.Series(pd.date_range(start=f'{yr}-01-01', end=str(end)[0:10], freq='D'), 
                           name='time'))
    else:
        times.append(pd.Series(pd.date_range(start=f'{yr}-01-01', end=f'{yr}-12-31', freq='D'), 
                           name='time'))
    

for i in range(len(ncdts)):
    lons = nc['longitude']
    lats = nc['latitude']
    time = ncdts[i]
    dat = nc.sel(time=time)
    arr = dat[list(dat.data_vars)[0]]
    arr_kron = np.kron(arr, np.ones((len(times[i]), 1, 1))).astype(np.int8)

    ncs.append(xr.Dataset({list(dat.data_vars)[0]:(("time","latitude" ,"longitude"), arr_kron)},
                 coords={
        "time": times[i],
        "latitude": lats,
        "longitude": lons,
        
    }))


In [37]:
ncs = xr.merge(ncs)

In [40]:
ncs['LC'].values = np.round(ncs['LC'].values, 0).astype(np.int8)

In [42]:
ncs.to_netcdf('./LC nc/LC.nc')